
**2-Engenharia-de-Dados-MVP-Criação-Carga-Qualidade-Silver**

Este notebook cria e carrega as tabelas da camada silver a partir das tabelas da camada bronze. Para criação das tabelas "escolas" e "cursos" da camada silver, campos das tabelas correspondentes na camada bronze são selecionados; seus tipos de dados são ajustados; valores texto, booleanos e nulos são tratados. 

Além disso, verifica-se que dados de unidade de ensino, uf e município presentes em cursos são das unidades de ensino. Por fim, são realizadas algumas análises que mostram que nem todas as unidades de ensino da tabela "cursos" existem na tabela "escolas" e, também, que nem todas as unidades de ensino da tabela "escolas" estão na tabela "cursos". 

In [0]:
%sql

-- 2) Criando camada silver

-- Criando camada silver
CREATE DATABASE silver


In [0]:
%sql

-- 3) Criando tabela escolas com campos selecionados e novos tipos de dados

CREATE TABLE silver.escolas (
    NOME_SISTEMA_DE_ENSINO VARCHAR(255),
    NOME_TIPO_ESCOLA VARCHAR(255),
    CODIGO_UNIDADE_DE_ENSINO INT,
    UNIDADE_DE_ENSINO VARCHAR(255),
    NOME_DEPENDENCIA_ADMINISTRATIVA VARCHAR(255),
    NOME_SUBDEPENDENCIA_ADMINISTRATIVA VARCHAR(255),
    UF CHAR(2),
    CODIGO_MUNICIPIO INT,
    MUNICIPIO VARCHAR(255)
)

In [0]:
%sql

-- 4) Povoando tabela escolas com dados tratados da camada bronze

INSERT INTO silver.escolas (NOME_SISTEMA_DE_ENSINO, NOME_TIPO_ESCOLA, CODIGO_UNIDADE_DE_ENSINO, UNIDADE_DE_ENSINO, NOME_DEPENDENCIA_ADMINISTRATIVA, NOME_SUBDEPENDENCIA_ADMINISTRATIVA, UF, CODIGO_MUNICIPIO, MUNICIPIO)
SELECT NOME_SISTEMA_DE_ENSINO, NOME_TIPO_ESCOLA, int(CODIGO_UNIDADE_DE_ENSINO), UPPER(UNIDADE_DE_ENSINO), NOME_DEPENDENCIA_ADMINISTRATIVA, (CASE WHEN NOME_SUBDEPENDENCIA_ADMINISTRATIVA = 'null' THEN 'NA' ELSE NOME_SUBDEPENDENCIA_ADMINISTRATIVA END) AS NOME_SUBDEPENDENCIA_ADMINISTRATIVA_novo, UF, int(CODIGO_MUNICIPIO), UPPER(MUNICIPIO) FROM bronze.escolas_sistec

num_affected_rows,num_inserted_rows
22508,22508


In [0]:
%sql

-- 5) Criando tabela cursos com campos selecionados e novos tipos de dados

CREATE TABLE silver.cursos (
    NOME_SUBTIPO_DE_CURSOS VARCHAR(255),
    CODIGO_CURSO INT,
    CURSO VARCHAR(255),
    EIXO_TECNOLOGICO VARCHAR(255),
    MODALIDADE VARCHAR(100),
    CARGA_HORARIA_CURSO INT,
    SITUACAO_ATIVO BOOLEAN,
    CODIGO_UNIDADE_DE_ENSINO INT,
    UNIDADE_DE_ENSINO VARCHAR(255),
    UF CHAR(2),
    CODIGO_MUNICIPIO INT,
    MUNICIPIO VARCHAR(255)
);

In [0]:
%sql

-- 6) Povoando tabela cursos com dados tratados da camada bronze

INSERT INTO silver.cursos (NOME_SUBTIPO_DE_CURSOS, CODIGO_CURSO, CURSO, EIXO_TECNOLOGICO, MODALIDADE, CARGA_HORARIA_CURSO,SITUACAO_ATIVO, CODIGO_UNIDADE_DE_ENSINO, UNIDADE_DE_ENSINO, UF, CODIGO_MUNICIPIO, MUNICIPIO)
SELECT NOME_SUBTIPO_DE_CURSOS, int(CODIGO_CURSO), CURSO, EIXO_TECNOLOGICO, MODALIDADE, int(CARGA_HORARIA_CURSO), (CASE WHEN SITUACAO_ATIVO = 't' THEN CAST('TRUE' AS BOOLEAN) ELSE CAST('FALSE' AS BOOLEAN) END) AS SITUACAO_ATIVO_novo, int(CODIGO_UNIDADE_DE_ENSINO), UPPER(UNIDADE_DE_ENSINO), UF, int(CODIGO_MUNICIPIO), UPPER(MUNICIPIO) FROM bronze.cursos_sistec

num_affected_rows,num_inserted_rows
83150,83150


In [0]:
%sql

-- 7) Avaliando campos em comum entre tabelas cursos e escolas - UNIDADE_DE_ENSINO

SELECT * FROM silver.cursos sc
INNER JOIN silver.escolas se ON sc.CODIGO_UNIDADE_DE_ENSINO = se.CODIGO_UNIDADE_DE_ENSINO
WHERE sc.UNIDADE_DE_ENSINO <> se.UNIDADE_DE_ENSINO

-- Resultado: Unidade de Ensino da tabela de cursos (silver) é a mesma da tabela de escolas (silver)


NOME_SUBTIPO_DE_CURSOS,CODIGO_CURSO,CURSO,EIXO_TECNOLOGICO,MODALIDADE,CARGA_HORARIA_CURSO,SITUACAO_ATIVO,CODIGO_UNIDADE_DE_ENSINO,UNIDADE_DE_ENSINO,UF,CODIGO_MUNICIPIO,MUNICIPIO,NOME_SISTEMA_DE_ENSINO,NOME_TIPO_ESCOLA,CODIGO_UNIDADE_DE_ENSINO,UNIDADE_DE_ENSINO,NOME_DEPENDENCIA_ADMINISTRATIVA,NOME_SUBDEPENDENCIA_ADMINISTRATIVA,UF,CODIGO_MUNICIPIO,MUNICIPIO


In [0]:
%sql

-- 8) Avaliando campos em comum entre tabelas cursos e escolas - UF

SELECT * FROM silver.cursos sc
INNER JOIN silver.escolas se ON sc.CODIGO_UNIDADE_DE_ENSINO = se.CODIGO_UNIDADE_DE_ENSINO
WHERE sc.UF <> se.UF

-- Resultado: UF da tabela de cursos (silver) é o mesmo da tabela de escolas (silver)

NOME_SUBTIPO_DE_CURSOS,CODIGO_CURSO,CURSO,EIXO_TECNOLOGICO,MODALIDADE,CARGA_HORARIA_CURSO,SITUACAO_ATIVO,CODIGO_UNIDADE_DE_ENSINO,UNIDADE_DE_ENSINO,UF,CODIGO_MUNICIPIO,MUNICIPIO,NOME_SISTEMA_DE_ENSINO,NOME_TIPO_ESCOLA,CODIGO_UNIDADE_DE_ENSINO,UNIDADE_DE_ENSINO,NOME_DEPENDENCIA_ADMINISTRATIVA,NOME_SUBDEPENDENCIA_ADMINISTRATIVA,UF,CODIGO_MUNICIPIO,MUNICIPIO


In [0]:
%sql

-- 9) Avaliando campos em comum entre tabelas cursos e escolas - MUNICIPIO

SELECT * FROM silver.cursos sc
INNER JOIN silver.escolas se ON sc.CODIGO_UNIDADE_DE_ENSINO = se.CODIGO_UNIDADE_DE_ENSINO
WHERE sc.MUNICIPIO <> se.MUNICIPIO

-- Resultado: Município da tabela de cursos (silver) é o mesmo da tabela de escolas (silver)

NOME_SUBTIPO_DE_CURSOS,CODIGO_CURSO,CURSO,EIXO_TECNOLOGICO,MODALIDADE,CARGA_HORARIA_CURSO,SITUACAO_ATIVO,CODIGO_UNIDADE_DE_ENSINO,UNIDADE_DE_ENSINO,UF,CODIGO_MUNICIPIO,MUNICIPIO,NOME_SISTEMA_DE_ENSINO,NOME_TIPO_ESCOLA,CODIGO_UNIDADE_DE_ENSINO,UNIDADE_DE_ENSINO,NOME_DEPENDENCIA_ADMINISTRATIVA,NOME_SUBDEPENDENCIA_ADMINISTRATIVA,UF,CODIGO_MUNICIPIO,MUNICIPIO


In [0]:
%sql

-- 10) Avaliando campos em comum entre tabelas cursos e escolas - CODIGO_MUNICIPIO

SELECT * FROM silver.cursos sc
INNER JOIN silver.escolas se ON sc.CODIGO_UNIDADE_DE_ENSINO = se.CODIGO_UNIDADE_DE_ENSINO
WHERE sc.CODIGO_MUNICIPIO <> se.CODIGO_MUNICIPIO

-- Resultado: Código do Município da tabela de cursos (silver) é o mesmo da tabela de escolas (silver)

NOME_SUBTIPO_DE_CURSOS,CODIGO_CURSO,CURSO,EIXO_TECNOLOGICO,MODALIDADE,CARGA_HORARIA_CURSO,SITUACAO_ATIVO,CODIGO_UNIDADE_DE_ENSINO,UNIDADE_DE_ENSINO,UF,CODIGO_MUNICIPIO,MUNICIPIO,NOME_SISTEMA_DE_ENSINO,NOME_TIPO_ESCOLA,CODIGO_UNIDADE_DE_ENSINO,UNIDADE_DE_ENSINO,NOME_DEPENDENCIA_ADMINISTRATIVA,NOME_SUBDEPENDENCIA_ADMINISTRATIVA,UF,CODIGO_MUNICIPIO,MUNICIPIO


In [0]:
%sql

-- 11) Avaliando se todas as unidades de ensino da tabela cursos estão na tabela escolas

SELECT count(*) FROM silver.cursos where CODIGO_UNIDADE_DE_ENSINO NOT IN (SELECT CODIGO_UNIDADE_DE_ENSINO FROM silver.escolas)

-- Resultado: 13 unidades de ensino da tabela cursos (silver) não foram encontradas na tabela escolas (silver) e, portanto, não possuem informações complementares próprias da tabela escolas


count(1)
13


In [0]:
%sql

-- 12) Avaliando se todas as unidades de ensino da tabela escolas estão na tabela cursos

SELECT count(*) FROM silver.escolas where CODIGO_UNIDADE_DE_ENSINO NOT IN (SELECT CODIGO_UNIDADE_DE_ENSINO FROM silver.cursos)

-- Resultado: 8.093 unidades de ensino da tabela escolas (silver) não foram encontradas na tabela cursos (silver) e, portanto, não possuem cursos relacionados


count(1)
8093
